<a href="https://colab.research.google.com/github/lmcanavals/topics/blob/main/0902_THe_zebra_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zebra puzzle
* There are five houses.
* The English man lives in the red house.
* The Swede has a dog.
* The Dane drinks tea.
* The green house is immediately to the left of the white house.
* They drink coffee in the green house.
* The man who smokes Pall Mall has birds.
* In the yellow house they smoke Dunhill.
* In the middle house they drink milk.
* The Norwegian lives in the first house.
* The man who smokes Blend lives in the house next to the house with cats.
* In a house next to the house where they have a horse, they smoke Dunhill.
* The man who smokes Blue Master drinks beer.
* The German smokes Prince.
* The Norwegian lives next to the blue house.
* They drink water in a house next to the house where they smoke Blend.

In [1]:
!pip install ortools 1> /dev/null

In [2]:
from ortools.sat.python import cp_model

In [3]:
model = cp_model.CpModel()
# 1: English, 2: Swedish, 3: Dane, 4: German, 5: Norwegian
N = [model.NewIntVar(1, 5, f"N_{i}") for i in range(5)]
# Tea, Cofee, Milk, Beer, Water
B = [model.NewIntVar(1, 5, f"B_{i}") for i in range(5)]
# Dog, Birds, Cats, Horse, Zebra
P = [model.NewIntVar(1, 5, f"P_{i}") for i in range(5)]
# Red Green White, Blue, Yellow
C = [model.NewIntVar(1, 5, f"C_{i}") for i in range(5)]
# Pall Mall, Dunhill, Blue Master, Prince, Blend
L = [model.NewIntVar(1, 5, f"L_{i}") for i in range(5)]

In [4]:
model.AddAllDifferent(N)
model.AddAllDifferent(B)
model.AddAllDifferent(P)
model.AddAllDifferent(C)
model.AddAllDifferent(L)

In [5]:
def exists(model, x1, v1, x2, v2):
    ex = [model.NewBoolVar(f"e_{j}") for j in range(5)]
    for i in range(5):
        model.Add(x1[i] == v1).OnlyEnforceIf(ex[i])
        model.Add(x1[i] != v1).OnlyEnforceIf(ex[i].Not())
        model.Add(x2[i] == v2).OnlyEnforceIf(ex[i])
        model.Add(x2[i] != v2).OnlyEnforceIf(ex[i].Not())
    model.Add(sum(ex) == 1)

In [6]:
def neighbor(model, x1, v1, x2, v2):
    n = [model.NewBoolVar(f"n_{j}") for j in range(4)]
    for i in range(4):
        b = [model.NewBoolVar(f"b_{j}") for j in range(2)]
        model.Add(x1[i] == v1).OnlyEnforceIf(b[0])
        model.Add(x1[i] != v1).OnlyEnforceIf(b[0].Not())
        model.Add(x2[i+1] == v2).OnlyEnforceIf(b[1])
        model.Add(x2[i+1] != v2).OnlyEnforceIf(b[1].Not())
        model.Add(sum(b) == 2).OnlyEnforceIf(n[i])
        model.Add(sum(b) != 2).OnlyEnforceIf(n[i].Not())
    return n

* The English man lives in the red house.

In [7]:
exists(model, N, 1, C, 1)

* The Swede has a dog.


In [8]:
exists(model, N, 2, P, 1)

* The Dane drinks tea.

In [9]:
exists(model, N, 3, B, 1)

* The green house is immediately to the left of the white house.


In [10]:
model.Add(sum(neighbor(model, C, 2, C, 3)) == 1)

* They drink coffee in the green house.


In [11]:
exists(model, B, 2, C, 2)

* The man who smokes Pall Mall has birds.


In [12]:
exists(model, L, 1, P, 2)

* In the yellow house they smoke Dunhill.


In [13]:
exists(model, C, 5, L, 2)

* In the middle house they drink milk.


In [14]:
model.Add(B[2] == 3)

* The Norwegian lives in the first house.


In [15]:
model.Add(N[0] == 5)

* The man who smokes Blend lives in the house next to the house with cats.


In [16]:
model.Add(sum(neighbor(model, L, 5, P, 3)) +
          sum(neighbor(model, P, 3, L, 5)) == 1)

* In a house next to the house where they have a horse, they smoke Dunhill.


In [17]:
model.Add(sum(neighbor(model, P, 4, L, 2)) +
          sum(neighbor(model, L, 2, P, 4)) == 1)

* The man who smokes Blue Master drinks beer.


In [18]:
exists(model, L, 3, B, 4)

* The German smokes Prince.


In [19]:
exists(model, N, 4, L, 4)

* The Norwegian lives next to the blue house.


In [20]:
model.Add(sum(neighbor(model, N, 5, C, 4)) +
          sum(neighbor(model, C, 5, N, 5)) == 1)

* They drink water in a house next to the house where they smoke Blend.


In [21]:
model.Add(sum(neighbor(model, B, 5, L, 5)) +
          sum(neighbor(model, L, 5, B, 5)) == 1)


# IT'S SOLVING TIME

In [22]:
solver = cp_model.CpSolver()
status = solver.Solve(model)
if status == cp_model.OPTIMAL:
    print("N", [solver.Value(x) for x in N])
    print("B", [solver.Value(x) for x in B])
    print("P", [solver.Value(x) for x in P])
    print("C", [solver.Value(x) for x in C])
    print("L", [solver.Value(x) for x in L])
    Nz = ["English", "Swedish", "Dane", "German", "Norwegian"]
    Bz = ["Tea", "Coffee", "Milk", "Beer", "Water"]
    Pz = ["Dog", "Birds", "Cats", "Horse", "Zebra"]
    Cz = ["Red", "Green", "White", "Blue", "Yellow"]
    Lz = ["Pall Mall", "Dunhill", "Blue Master", "Prince", "Blend"]
    for i in range(5):
        print(f"In the house {i+1}:\t",
              f"{Nz[solver.Value(N[i]) - 1]}\t",
              f"{Bz[solver.Value(B[i]) - 1]}\t",
              f"{Pz[solver.Value(P[i]) - 1]}\t",
              f"{Cz[solver.Value(C[i]) - 1]}\t",
              f"{Lz[solver.Value(L[i]) - 1]}\t")
else:
    print(":(")

N [5, 3, 1, 4, 2]
B [5, 1, 3, 2, 4]
P [3, 4, 2, 5, 1]
C [5, 4, 1, 2, 3]
L [2, 5, 1, 4, 3]
In the house 1:	 Norwegian	 Water	 Cats	 Yellow	 Dunhill	
In the house 2:	 Dane	 Tea	 Horse	 Blue	 Blend	
In the house 3:	 English	 Milk	 Birds	 Red	 Pall Mall	
In the house 4:	 German	 Coffee	 Zebra	 Green	 Prince	
In the house 5:	 Swedish	 Beer	 Dog	 White	 Blue Master	
